In [24]:
import random
import pickle
import pennylane as qml
import torch
from data_utils import mnist_preparation
from typing import Optional, Dict, List, Any
from torch.utils.data import DataLoader, dataloader
from tqdm import tqdm
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from pennylane import numpy as np

In [25]:
labels = [0,1,2,3]
# Download MNIST and prepare transforms
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                                transforms.Resize((16, 16)),  # Resize to 16x16
                                transforms.ToTensor(),
                                #transforms.Lambda(lambda img: add_salt_and_pepper_noise(img, salt_prob=0.1, pepper_prob=0.1)),
                                transforms.Normalize((0.1307,), (0.3081,))  # Normalize
                             ]))
#train/vali/test 70/15/15 split, see data_utils.py for further details
train_dataloader, validation_dataloader, test_dataloader = mnist_preparation(dataset=mnist, labels = labels, train_test_ratio=0.7,batch_size=64, vali_test_ratio=0.5)

print("Images in the training set: ", len(train_dataloader.dataset), "\n Images in the validation set: ", len(validation_dataloader.dataset), "\n Images in the test set: ", len(test_dataloader.dataset))

Images in the training set:  17327 
 Images in the validation set:  3713 
 Images in the test set:  3714


In [43]:
with open("/Users/jackvittori/Desktop/4layerokkkk/weight4layer.pickle", "rb") as file: 
    weights = pickle.load(file)
for param in weights.values():
    param.requires_grad = False

In [30]:
import pickle
with open('/Users/jackvittori/Desktop/allenamento26sett/om/trhistory.pickle', 'rb') as file: 
    tr_history = pickle.load(file)
weights = tr_history['weights']

In [44]:
p = 0.001

def early_evaluation_utils(state: torch.Tensor = None): 
    first_pair = [0,1]
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(wires = first_pair)

In [45]:
mixed_device = qml.device("default.mixed", wires=[0,1,2,3,4,5,6,7,8,9], shots = 50)
early_qnode = qml.QNode(early_evaluation_utils, mixed_device)

In [46]:
result = []
for i, (img, label) in tqdm(enumerate(test_dataloader.dataset)):
    
    if i == 100:
        break        
    img = img.type(torch.float64)
    img = img / torch.linalg.norm(img).view(-1, 1) 
    probs = early_qnode(img, shots = 50)
    prediction = torch.argmax(probs)
    result.append((prediction, label))
     
def calculate_accuracy(data):
    correct = sum([1 for label, prediction in data if label == prediction])
    return correct, correct / len(data)

test_results = calculate_accuracy(result)

print(test_results[0], "elements have been correctly classified over", len(test_dataloader.dataset), "total images with an accuracy of ", test_results[1])

100it [07:30,  4.51s/it]

78 elements have been correctly classified over 3714 total images with an accuracy of  0.78


In [23]:
p = 0.03

def early_evaluation_utils(state: torch.Tensor = None): 
    first_pair = [0,1]
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))
    for i in range(4):
        for j in range(8):
            qml.RX(weights[f'layer_{i}'][j, 0], wires=j)
            qml.RY(weights[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(weights[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.DepolarizingChannel(p=p, wires=(j + 1) % 8)
    
    return qml.probs(wires = first_pair)

mixed_device = qml.device("default.mixed", wires=[0,1,2,3,4,5,6,7,8,9], shots = 50)
early_qnode = qml.QNode(early_evaluation_utils, mixed_device)
result = []
for i, (img, label) in tqdm(enumerate(test_dataloader.dataset)):
    
    if i == 100:
        break        
    img = img.type(torch.float64)
    img = img / torch.linalg.norm(img).view(-1, 1) 
    probs = early_qnode(img, shots = 50)
    prediction = torch.argmax(probs)
    result.append((prediction, label))
     
def calculate_accuracy(data):
    correct = sum([1 for label, prediction in data if label == prediction])
    return correct, correct / len(data)

test_results = calculate_accuracy(result)

print(test_results[0], "elements have been correctly classified over", len(test_dataloader.dataset), "total images with an accuracy of ", test_results[1])

100it [07:28,  4.48s/it]

60 elements have been correctly classified over 3714 total images with an accuracy of  0.6
